In [1]:
import sys
import torch
sys.path.append('../src')
%load_ext autoreload
%autoreload 2

from generator import *
from evaluation import *
from fair_model import FairModel
from baselines import LR, CvxFairModel, EOFairModel
from utils import gen_plot_data, plot_axes, combine_tuples

In [ ]:
# Generate data via run_simulation() and save a full panel CSV (includes neighbor decision vector D1..D10)
steps = 5

# run_simulation returns: s, adj, edges, Xs, Ys, Ds, Ps, Os, Us, As
s, adj, edges, Xs, Ys, Ds, Ps, Os, Us, As = run_simulation(
    decision_model=bank,
    repayment_model=bank,
    agent=agent_train,
    steps=steps,
    enforce_demographic_mixing=True,
    k_same=8,
    k_other=2,
    directed=False,
    graph_seed=2021,
    seed=2021,
    decision_coef=0.8,
    repayment_coef=0.8,
    exclude_inactive_neighbors_from_O_denominator=False,
 )

out_path = "../src/simulation_results_neighbors.csv"
save_agent_panel_csv(
    out_path,
    s=s,
    Xs=Xs,
    adj=adj,
    Ds=Ds,
    Ys=Ys,
    Ps=Ps,
    Us=Us,
    As=As,
    Os=Os,
    t0=0,
    neighbor_k=10,
 )

print("Wrote:", out_path)

TypeError: Agent.compute_observed_rejection_gap() got multiple values for argument 'active'